<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/8_Array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part of the free course [EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html). 99% of this material has been adapted from [Google Earth Engine Guides](https://developers.google.com/earth-engine/).

<!--NAVIGATION-->
 < [Chart](7_Chart.ipynb) | [Contents](index.ipynb) |  [Specialized Algorithms](9_SpecializedAlgorithms.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/8_Array.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<center>
<h1>Google Earth Engine with Python </h1>
<h2> Array </h2>
</center>
<h2> Topics:</h2>

1. Array Overview
2. Arrays and Array Images
3. Array Transformations
4. Eigen Analysis
5. Array Sorting and Reducing


## Connecting GEE with Google Services

- **Authenticate to Earth Engine**

In [0]:
!pip install earthengine-api #earth-engine Python API

In [0]:
!earthengine authenticate 

- **Authenticate to Google Drive (OPTIONAL)**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

- **Authenticate to Google Cloud (OPTIONAL)**

In [0]:
from google.colab import auth
auth.authenticate_user()

## Testing the software setup

In [0]:
# Earth Engine Python API
import ee
ee.Initialize()

In [0]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

In [0]:
#@title Mapdisplay: Display GEE Features or Images using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [0]:
#@title # Array Overview
from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/-qo8L5GmKO0" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

Earth Engine represents 1-D vectors, 2-D matrices, 3-D cubes, and higher dimensional hypercubes with the **ee.Array** type. Arrays are a flexible data structure, but in exchange for the power they offer, they do not scale as well as other data structures in Earth Engine. If the problem can be solved without using arrays, the result will be computed faster and more efficiently. But if the problem requires a higher dimension model, flexible linear algebra, or anything else arrays are uniquely suited to, you can use the **Array** class.

## Array dimension, shape and size

The dimension of an array refers to the number of axes along which the underlying data varies. For example, 0-D arrays are scalar numbers, 1-D arrays are vectors, 2-D arrays are matrices, 3-D arrays are cubes, and >3-D arrays are hyper-cubes. For an N-dimensional array, there are N axes from 0 to N-1. The shape of the array is determined by the lengths of the axes. The length of an axis is the number of positions along it. The array size, or number of total elements in the array, equals the product of the axis lengths. Each value at every position on every axis must have a valid number, since sparse or ragged arrays are not currently supported. The array’s element type indicates what kind of number each element is; all elements of the array will have the same type.

# Arrays and Array Images

Arrays in Earth Engine are constructed from lists of numbers and lists of lists. The degree of nesting determines the number of dimensions. To get started with a simple, motivated example, consider the following example of an Array created from Landsat 5 tasseled cap (TC) coefficients ([Crist and Cicone 1984](http://dx.doi.org/10.1109/TGRS.1984.350619))

In [0]:
# Create an Array of Tasseled Cap coefficients.
coefficients = ee.Array([
  [0.3037, 0.2793, 0.4743, 0.5585, 0.5082, 0.1863],
  [-0.2848, -0.2435, -0.5436, 0.7243, 0.0840, -0.1800],
  [0.1509, 0.1973, 0.3279, 0.3406, -0.7112, -0.4572],
  [-0.8242, 0.0849, 0.4392, -0.0580, 0.2012, -0.2768],
  [-0.3280, 0.0549, 0.1075, 0.1855, -0.4357, 0.8085],
  [0.1084, -0.9022, 0.4120, 0.0573, -0.0251, 0.0238]
])

Confirm that this is a 6x6, 2-D Array using length(), which will return the lengths of each axis.

In [0]:
# Print the dimensions.
print(coefficients.length())
# [6,6]

The following table illustrates the arrangement of the matrix entries along the 0-axis and the 1-axis.

<tbody><tr>
        <th></th><th></th><th colspan="6">1-axis -&gt;</th>
      </tr>
      <tr class="alt">
        <td></td><td></td><td><b>0</b></td><td><b>1</b></td><td><b>2</b></td><td><b>3</b></td><td><b>4</b></td><td><b>5</b></td>
      </tr>
      <tr>
        <td></td><td><b>0</b></td><td>0.3037</td><td>0.2793</td><td>0.4743</td><td>0.5585</td><td>0.5082</td><td>0.1863</td>
      </tr>
      <tr>
        <td></td><td><b>1</b></td><td>-0.2848</td><td>-0.2435</td><td>-0.5436</td><td>0.7243</td><td>0.0840</td><td>-0.1800</td>
      </tr>
      <tr>
        <td><b>0-axis</b></td><td><b>2</b></td><td>0.1509</td><td>0.1973</td><td>0.3279</td><td>0.3406</td><td>-0.7112</td><td>-0.4572</td>
      </tr>
      <tr>
        <td></td><td><b>3</b></td><td>-0.8242</td><td>0.0849</td><td>0.4392</td><td>-0.0580</td><td>0.2012</td><td>-0.2768</td>
      </tr>
      <tr>
        <td></td><td><b>4</b></td><td>-0.3280</td><td>0.0549</td><td>0.1075</td><td>0.1855</td><td>-0.4357</td><td>0.8085</td>
      </tr>
      <tr>
        <td></td><td><b>5</b></td><td>0.1084</td><td>-0.9022</td><td>0.4120</td><td>0.0573</td><td>-0.0251</td><td>0.0238</td>
      </tr>
    </tbody>

The indices on the left of the table indicate positions along the 0-axis. The n-th element within each list on the 0-axis is in the n-th position along the 1-axis. For example, the entry at coordinate [3,1] of the array is 0.0849. Suppose ‘greenness’ is the TC component of interest. You can get the greenness sub-matrix using slice().

In [0]:
# Get the 1x6 greenness slice, display it.
greenness = coefficients.slice(axis=0, start=1, end=2, step=1)
print(greenness)

The 2-D greenness matrix should look something like.

**[[-0.2848, -0.2435, -0.5436, 0.7243, 0.084, -0.18 ]]**

Observe that the **start** and end parameters of **slice()** correspond to the 0-axis indices displayed in the table (**start** is inclusive and end is exclusive).

To get a greenness image, matrix multiply the bands of a Landsat 5 image by the greenness matrix. To do that, first convert the multi-band Landsat image into an “Array Image”, where each pixel is an **Array** of band values.

In [0]:
# Load a Landsat 5 image, select the bands of interest.
image = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_044034_20081011').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])

# Make an Array Image, with a 1-D Array per pixel.
arrayImage1D = image.toArray()

# Make an Array Image with a 2-D Array per pixel, 6x1.
arrayImage2D = arrayImage1D.toArray(1)

In this example, note that **toArray()** converts **image** to an array image in which each pixel is a 1-D vector, the entries of which correspond to the 6 values at the corresponding positions in the bands of **image**. An array image of 1-D vectors created in this manner has no concept of 2-D shape. To perform 2-D only operations such as matrix multiplication, convert it into a 2-D array per-pixel image with **toArrray(1)**. In each pixel of the 2-D array image, there is a 6x1 matrix of band values. To see this, consider the following toy example.

In [0]:
array1D = ee.Array([1, 2, 3])            # [1,2,3]
array2D = ee.Array.cat([array1D], 1)     # [[1],[2],[3]]

Observe that the **array1D** vector varies along the 0-axis. The **array2D** matrix does as well, but it’s got an extra dimension. Calling **toArray(1)** on the array image is like calling cat(**bandVector**, 1) on every pixel. Using the 2-D array image, left multiply by an image where each pixel contains a 2-D matrix of greenness coefficients.

In [0]:
# Do a matrix multiplication: 1x6 times 6x1.
# Cast the greenness Array to an Image prior to multiplication.
greennessArrayImage = ee.Image(greenness).matrixMultiply(arrayImage2D)

The result is a new array image where every pixel is the 1x1 matrix that results from matrix multiplying the 1x6 greenness matrix (left) and the 6x1 band matrix (right). For display purposes, convert to a regular, one-band image with **arrayGet()**.

In [0]:
# Get the result from the 1x1 array in each pixel of the 2-D array image.
greennessImage = greennessArrayImage.arrayGet([0, 0])

# Display the input imagery with the greenness result.
dicc = {
    'image': image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.5}),
    'greenness': greennessImage.getMapId({'min': -0.1, 'max': 0.1})
}

coords = [37.562, -122.3]
Mapdisplay(coords, dicc, "Stamen Terrain", 10)

Here is a complete example, which uses the entire coefficients array to compute multiple tasseled cap components at once and display the result.

In [0]:
# Define an Array of Tasseled Cap coefficients.
coefficients = ee.Array([
  [0.3037, 0.2793, 0.4743, 0.5585, 0.5082, 0.1863],
  [-0.2848, -0.2435, -0.5436, 0.7243, 0.0840, -0.1800],
  [0.1509, 0.1973, 0.3279, 0.3406, -0.7112, -0.4572],
  [-0.8242, 0.0849, 0.4392, -0.0580, 0.2012, -0.2768],
  [-0.3280, 0.0549, 0.1075, 0.1855, -0.4357, 0.8085],
  [0.1084, -0.9022, 0.4120, 0.0573, -0.0251, 0.0238]
])

# Load a Landsat 5 image, select the bands of interest.
image = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_044034_20081011').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])

# Make an Array Image, with a 1-D Array per pixel.
arrayImage1D = image.toArray()

# Make an Array Image with a 2-D Array per pixel, 6x1.
arrayImage2D = arrayImage1D.toArray(1)

# Do a matrix multiplication: 6x6 times 6x1.
# Get rid of the extra dimensions.
componentsImage = ee.Image(coefficients).matrixMultiply(arrayImage2D).arrayProject([0]).arrayFlatten([['brightness', 'greenness', 'wetness', 'fourth', 'fifth', 'sixth']])

# Display the first three bands of the result and the input imagery.
vizParams = {
  'bands': ['brightness', 'greenness', 'wetness'],
  'min': -0.1, 'max': [0.5, 0.1, 0.1]
}

# Display the input imagery with the greenness result.
dicc = {
    'image': image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.5}),
    'components': componentsImage.getMapId(vizParams)
}

coords = [37.562, -122.3]
Mapdisplay(coords, dicc, "Stamen Terrain", 10)

# Tasseled cap components “brightness” (red), “greenness” (green), and “wetness” (blue).

Note that when getting bands from an array image, first get rid of the extra dimensions with **project()**, then convert it back to a regular image with **arrayFlatten()**. The output should look something like.

# Array Transformations

Earth Engine supports array transformations such as transpose, inverse and pseudo-inverse. As an example, consider an ordinary least squares (OLS) regression of a time series of images. In the following example, an image with bands for predictors and a response is converted to an array image, then “solved” to obtain least squares coefficients estimates three ways. First, assemble the image data and convert to arrays.

In [0]:
# This function masks the input with a threshold on the simple cloud score.
def cloudMask (img):
  cloudscore = ee.Algorithms.Landsat.simpleCloudScore(img).select('cloud')
  return img.updateMask(cloudscore.lt(50))

# Load a Landsat 5 image collection.
  # Filter to get only two years of data.
  # Filter to get only imagery at a point of interest.
  # Mask clouds by mapping the cloudMask function over the collection.
  # Select NIR and red bands only.
  # Sort the collection in chronological order.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA').filterDate('2008-04-01', '2010-04-01').filterBounds(ee.Geometry.Point(-122.2627, 37.8735)).map(cloudMask).select(['B4', 'B3']).sort('system:time_start', True)

# This function computes the predictors and the response from the input.
  # Compute time of the image in fractional years relative to the Epoch.
  # Compute the season in radians, one cycle per year.
  # Return an image of the predictors followed by the response.
  # 0. constant
  # 1. linear trend
  # 2. seasonal
  # 3. seasonal
  # 4. response
def makeVariables(image):
  year = ee.Image(image.date().difference(ee.Date('1970-01-01'), 'year'))
  season = year.multiply(2 * math.pi)
  return image.select().addBands(ee.Image(1)).addBands(year.rename('t')).addBands(season.sin().rename('sin')).addBands(season.cos().rename('cos')).addBands(image.normalizedDifference().rename('NDVI')).toFloat()

# Define the axes of variation in the collection array.
imageAxis = 0
bandAxis = 1

# Convert the collection to an array.
array = collection.map(makeVariables).toArray()

# Check the length of the image axis (number of images).
arrayLength = array.arrayLength(imageAxis)
# Update the mask to ensure that the number of images is greater than or
# equal to the number of predictors (the linear model is solveable).
array = array.updateMask(arrayLength.gt(4))

# Get slices of the array according to positions along the band axis.
predictors = array.arraySlice(bandAxis, 0, 4)
response = array.arraySlice(bandAxis, 4)

Note that **arraySlice()** returns all the images in the time series for the range of indices specified along the **bandAxis** (the 1-axis). At this point, matrix algebra can be used to solve for the OLS coefficients.

In [0]:
# Compute coefficients the hard way.
coefficients1 = predictors.arrayTranspose().matrixMultiply(predictors).matrixInverse().matrixMultiply(predictors.arrayTranspose()).matrixMultiply(response)

Although this method works, it is inefficient and makes for difficult to read code. A better way is to use the **pseudoInverse()** method (**matrixPseudoInverse()** for an array image).

In [0]:
# Compute coefficients the easy way.
coefficients2 = predictors.matrixPseudoInverse().matrixMultiply(response)
    

From a readability and computational efficiency perspective, the best way to get the OLS coefficients is **solve()** (**matrixSolve()** for an array image). The **solve()** function determines how to best solve the system from characteristics of the inputs, using the pseudo-inverse for overdetermined systems, the inverse for square matrices and special techniques for nearly singular matrices:



In [0]:
# Compute coefficients the easiest way.
coefficients3 = predictors.matrixSolve(response)

To get a multi-band image, project the array image into a lower dimensional space, then flatten it.

In [0]:
# Turn the results into a multi-band image.
  # Get rid of the extra dimensions.
coefficientsImage = coefficients3.arrayProject([0]).arrayFlatten([['constant', 'trend', 'sin', 'cos']])

Examine the outputs of the three methods and observe that the resultant matrix of coefficients is the same regardless of the solver. That **solve()** is flexible and efficient makes it a good choice for general purpose linear modeling.

# Eigen Analysis

The [principal components (PC) transform](https://en.wikipedia.org/wiki/Principal_component_analysis)  (also known as the Karhunen-Loeve transform) is a spectral rotation that takes spectrally correlated image data and outputs uncorrelated data. The PC transform accomplishes this by diagonalizing the input band correlation matrix through Eigen-analysis. To do this in Earth Engine, use a covariance reducer on an array image and the **eigen()** command on the resultant covariance array. Consider the following function for that purpose (which is part of a complete example):

In [0]:
def getPrincipalComponents(centered, scale, region):
  # Collapse the bands of the image into a 1D array per pixel.
  arrays = centered.toArray()
  # Compute the covariance of the bands within the region.
  co= arrays.reduceRegion({
    reducer: ee.Reducer.centeredCovariance(),
    geometry: region,
    scale: scale,
    maxPixels: 1e9
  })

  # Get the 'array' covariance result and cast to an array.
  # This represents the band-to-band covariance within the region.
  covarArray = ee.Array(covar.get('array'))

  # Perform an eigen analysis and slice apart the values and vectors.
  eigens = covarArray.eigen()

  # This is a P-length vector of Eigenvalues.
  eigenValues = eigens.slice(1, 0, 1)
  # This is a PxP matrix with eigenvectors in rows.
  eigenVectors = eigens.slice(1, 1)

  # Convert the array image to 2D arrays for matrix computations.
  arrayImage = arrays.toArray(1)

  # Left multiply the image array by the matrix of eigenvectors.
  principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

  # Turn the square roots of the Eigenvalues into a P-band image.
  sdImage = ee.Image(eigenValues.sqrt()).arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

  # Turn the PCs into a P-band image, normalized by SD.
      # Throw out an an unneeded dimension, [[]] -> [].
      # Make the one band array image a multi-band image, [] -> image.
      # Normalize the PCs by their SDs.
  return principalComponents.arrayProject([0]).arrayFlatten([getNewBandNames('pc')]).divide(sdImage)


The input to the function is a mean zero image, a scale and a region over which to perform the analysis. Note that the input imagery first needs to be converted to a 1-D array image and then reduced using **ee.Reducer.centeredCovariance()**. The array returned by this reduction is the symmetric variance-covariance matrix of the input. Use the **eigen()** command to get the eigenvalues and eigenvectors of the covariance matrix. The matrix returned by **eigen()** contains the eigenvalues in the 0-th position of the 1-axis. As shown in the above function, use **slice()** to separate the eigenvalues and the eigenvectors. Each element along the 0-axis of the eigenVectors matrix is an eigenvector. As in the tasseled cap (TC) example, perform the transformation by matrix multiplying the **arrayImage** by the eigenvectors. In this example, each eigenvector multiplication results in a PC.

# Array Sorting and Reducing

Array sorting is useful for obtaining custom quality mosaics which involve reducing a subset of image bands according to the values in a different band. The following example sorts by a cloud index, then gets the mean of the least cloudy subset of images in the collection.

In [0]:
# Define an arbitrary region of interest as a point.
roi = ee.Geometry.Point(-122.26032, 37.87187)

# Use these bands.
bandNames = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11'])

# Load a Landsat 8 collection.
  # Select the bands of interest to avoid taking up memory.
  # Filter to get only imagery at a point of interest.
  # Filter to get only six months of data.
  # Mask clouds by mapping the cloudMask function over the collection.
  # This will add a cloud score band called 'cloud' to every image.

def simpleCloudImage(image):
  return ee.Algorithms.Landsat.simpleCloudScore(image)

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').select(bandNames).filterBounds(roi).filterDate('2014-06-01', '2014-12-31').map(ee.Algorithms.Landsat.simpleCloudScore)

# Convert the collection to an array.
array = collection.toArray()

# Label of the axes.
imageAxis = 0
bandAxis = 1

# Get the cloud slice and the bands of interest.
bands = array.arraySlice(bandAxis, 0, bandNames.length())
clouds = array.arraySlice(bandAxis, bandNames.length())

# Sort by cloudiness.
sorted = bands.arraySort(clouds)

# Get the least cloudy images, 20% of the total.
numImages = sorted.arrayLength(imageAxis).multiply(0.2).int()
leastCloudy = sorted.arraySlice(imageAxis, 0, numImages)

# Get the mean of the least cloudy images by reducing along the image axis.
mean = leastCloudy.arrayReduce(
  reducer= ee.Reducer.mean(),
  axes= [imageAxis]
)

# Turn the reduced array image into a multi-band image for display.
meanImage = mean.arrayProject([bandAxis]).arrayFlatten([bandNames])

# Display the input imagery with the greenness result.
dicc = {
    'roi': roi.getInfo(),
    'meanImage': meanImage.getMapId({'bands': ['B5', 'B4', 'B2'], 'min': 0, 'max': 0.5})
}

coords = [37.562, -122.3]
Mapdisplay(coords, dicc, "Stamen Terrain", 9)

As in the linear modeling example, separate the bands of interest from the sort index using **arraySlice()** along the band axis. Then sort the bands of interest by the cloud index using **arraySort()**. After the pixels have been sorted by ascending cloudiness, use **arraySlice()** along the **imageAxis** to get 20% of the least cloudy pixels. Lastly, apply **arrayReduce()** along the **imageAxis** with a mean reducer to get the mean of the least cloudy pixels. The final step converts the array image back to a multi-band image for display.


<!--NAVIGATION-->
 < [Chart](7_Chart.ipynb) | [Contents](index.ipynb) |  [Specialized Algorithms](9_SpecializedAlgorithms.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/8_Array.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>